In [2]:
import torch
from pytorch_quantization import tensor_quant

from torch import nn, outer

from pytorch_quantization import tensor_quant
import pytorch_quantization.nn as quant_nn
import pytorch_quantization
pytorch_quantization.__version__
from tqdm import tqdm


In [3]:
from datasets.yolo import YOLO
from engine.detector import Detector
from model.squeezedet import SqueezeDet, SqueezeDetWithLoss
from utils.config import Config
from utils.model import load_model
from PIL import Image
import numpy as np
import argparse
import os
from utils.misc import init_env

In [4]:
args = {'_': '_'}
cfg = Config().parse(args)

init_env(cfg) 
 
cfg.mode = 'eval'
cfg.exp_id = "test_veh_det_trt_calib_sqdet"
cfg.dataset = 'yolo'
cfg.load_model = '../exp/model_41.pth' 
cfg.batch_size = 1
cfg.arch = 'squeezedet'
cfg.num_workers = 0
cfg.save_intervals = 1
# cfg.stride = 8
cfg.debug = 0
cfg.device = 'cuda'
cfg.load_pretrained = True

def update_exp_dir(cfg, exp_id):
    cfg.save_dir = os.path.join(cfg.exp_dir, exp_id)
    os.makedirs(cfg.save_dir, exist_ok=True)
    cfg.log_file = os.path.join(cfg.save_dir, 'training_logs.txt')
    os.remove(cfg.log_file) if os.path.exists(cfg.log_file) else None
    cfg.debug_dir = os.path.join(cfg.save_dir, 'debug')

update_exp_dir(cfg, cfg.exp_id)

Config().print(cfg)


arch                           squeezedet
batch_size                     1
bbox_loss_weight               20.0
chunk_sizes                    [32]
class_loss_weight              1.0
data_dir                       /home/hazen/data/datasets
dataset                        yolo
debug                          0
debug_dir                      /home/hazen/data/SqueezeDet-PyTorch/exp/test_veh_det_trt_calib_sqdet/debug
device                         cuda
drift_prob                     1.0
dropout_prob                   0
exp_dir                        /home/hazen/data/SqueezeDet-PyTorch/exp
exp_id                         test_veh_det_trt_calib_sqdet
flip_prob                      0.5
forbid_resize                  False
gpus                           [0]
gpus_str                       0
grad_norm                      0.5
keep_top_k                     64
load_model                     ../exp/model_41.pth
load_pretrained                True
log_file                       /home/hazen/data/Squeeze

In [4]:
# from eval import eval
# eval(cfg)

In [5]:
from engine.detector import Detector
from model.squeezedet import SqueezeDetWithLoss
from utils.model import load_model
from utils.misc import load_dataset



In [6]:
dataset = load_dataset(cfg.dataset)('val', cfg)
cfg = Config().update_dataset_info(cfg, dataset)
Config().print(cfg)


anchors                        [[  8.   8.  32.  20.]
 [  8.   8.  61.  42.]
 [  8.   8.  59.  97.]
 ...
 [440. 248. 160. 152.]
 [440. 248. 211. 201.]
 [440. 248. 343. 205.]]
anchors_per_grid               9
arch                           squeezedet
batch_size                     1
bbox_loss_weight               20.0
chunk_sizes                    [32]
class_loss_weight              1.0
class_names                    ('bike', 'car', 'bus')
data_dir                       /home/hazen/data/datasets
dataset                        yolo
debug                          0
debug_dir                      /home/hazen/data/SqueezeDet-PyTorch/exp/test_veh_det_trt_calib_sqdet/debug
device                         cuda
drift_prob                     1.0
dropout_prob                   0
exp_dir                        /home/hazen/data/SqueezeDet-PyTorch/exp
exp_id                         test_veh_det_trt_calib_sqdet
flip_prob                      0.5
forbid_resize                  False
gpus             

In [6]:
print (dataset.phase)

val


In [7]:
if cfg.mode=='eval':
        model = SqueezeDetWithLoss(cfg)
        model = load_model(model, cfg.load_model, cfg)


detect = model.detect
model.detect = True
detector = Detector(model, cfg)
print(detector.model)

loaded model ../exp/model_41.pth, epoch 41
state_dict: 
 dict_keys(['base.conv1.weight', 'base.conv1.bias', 'base.features.1.squeeze.weight', 'base.features.1.squeeze.bias', 'base.features.1.expand1x1.weight', 'base.features.1.expand1x1.bias', 'base.features.1.expand3x3.weight', 'base.features.1.expand3x3.bias', 'base.features.2.squeeze.weight', 'base.features.2.squeeze.bias', 'base.features.2.expand1x1.weight', 'base.features.2.expand1x1.bias', 'base.features.2.expand3x3.weight', 'base.features.2.expand3x3.bias', 'base.features.4.squeeze.weight', 'base.features.4.squeeze.bias', 'base.features.4.expand1x1.weight', 'base.features.4.expand1x1.bias', 'base.features.4.expand3x3.weight', 'base.features.4.expand3x3.bias', 'base.features.5.squeeze.weight', 'base.features.5.squeeze.bias', 'base.features.5.expand1x1.weight', 'base.features.5.expand1x1.bias', 'base.features.5.expand3x3.weight', 'base.features.5.expand3x3.bias', 'base.features.7.squeeze.weight', 'base.features.7.squeeze.bias', 'b

In [8]:
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import calib
from pytorch_quantization.tensor_quant import QuantDescriptor

from absl import logging
logging.set_verbosity(logging.INFO)  # Disable logging as they are too noisy in notebook

In [9]:
quant_desc_input = QuantDescriptor(calib_method='histogram')
quant_nn.QuantConv2d.set_default_quant_desc_input(quant_desc_input)
quant_nn.QuantLinear.set_default_quant_desc_input(quant_desc_input)

In [31]:
from pytorch_quantization import quant_modules
quant_modules.initialize()

In [8]:
print(detector.model)

SqueezeDetWithLoss(
  (base): SqueezeDetBase(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu1): ReLU(inplace=True)
    (features): Sequential(
      (0): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      (1): Fire(
        (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
        (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
        (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation_1): ReLU(inplace=True)
        (activation_2): ReLU(inplace=True)
        (activation_3): ReLU(inplace=True)
        (float_functional_simple): FloatFunctional(
          (activation_post_process): Identity()
        )
      )
      (2): Fire(
        (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
        (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
        (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1,

### Calibration Functions

In [21]:
def collect_stats(model, data_loader, num_batches):
    """Feed data to the network and collect statistic"""

    # Enable calibrators
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.disable_quant()
                module.enable_calib()
            else:
                module.disable()

    # for i, (image, _) in tqdm(enumerate(data_loader), total=num_batches):
    #     model(image.cuda())
    #     if i >= num_batches:
    #         break
    # since yolo/lpr dataloader return a dict, so writing it in this way
    for i, (batch) in tqdm(enumerate(data_loader), total=num_batches):
        print(batch.keys())
        for k in batch:
                if 'image_meta' not in k:
                    batch[k] = batch[k].to(device='cuda', non_blocking=True)
        model(batch)
        if i >= num_batches:
            break

    # Disable calibrators
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.enable_quant()
                module.disable_calib()
            else:
                module.enable()
            
def compute_amax(model, **kwargs):
    # Load calib result
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                if isinstance(module._calibrator, calib.MaxCalibrator):
                    module.load_calib_amax()
                else:
                    module.load_calib_amax(**kwargs)
#             print(F"{name:40}: {module}")
    model.cuda()

## Make Dataset and DataLoader

In [8]:
import os
import subprocess

import numpy as np
import skimage.io

from datasets.base import BaseDataset
from utils.boxes import generate_anchors
from PIL import Image
from torchvision.datasets.folder import default_loader


class YOLO_CalibTRT(BaseDataset):
    def __init__(self, phase, cfg):
        super(YOLO_CalibTRT, self).__init__(phase, cfg)

        self.input_size = (256, 448)  # (height, width), both dividable by 16
        self.class_names = ('bike', 'car', 'bus')
        # real_filtered mean and std
        # self.rgb_mean = np.array([94.87347, 96.89165, 94.70493], dtype=np.float32).reshape(1, 1, 3)
        # self.rgb_std = np.array([53.869507, 53.936283, 55.2807], dtype=np.float32).reshape(1, 1, 3)
        
        # real_filtered plus all_sites_seatbelt mean and std
        self.rgb_mean = np.array([104.90631, 105.41336, 104.70162], dtype=np.float32).reshape(1, 1, 3)
        self.rgb_std = np.array([50.69564, 49.60443, 50.158844], dtype=np.float32).reshape(1, 1, 3)
        self.num_classes = len(self.class_names)
        self.class_ids_dict = {cls_name: cls_id for cls_id, cls_name in enumerate(self.class_names)}

        self.data_dir = os.path.join(cfg.data_dir, 'all_real_plus_synth_8sites_plus_SVsynth_plus_seatbelt_plus_new_trajectory_data_kitti_format_5percentofwidth_filtered')
        self.sample_ids, self.sample_set_path = self.get_sample_ids()

        self.grid_size = tuple(x //cfg.stride  for x in self.input_size)  # anchors grid 
        # self.anchors_seed = np.array([[ 29, 17], [46, 32], [69, 52],
        #                                 [109, 68], [84, 127], [155, 106], 
        #                                 [255, 145], [183, 215], [371, 221]], dtype=np.float32) ## real_filtered anchors
        
        self.anchors_seed = np.array( [[ 32, 20], [ 61, 42], [ 59, 97],
                                        [103, 66], [122, 114], [183, 96],
                                        [160, 152], [211, 201], [343, 205]], dtype=np.float32) ## real_filtered plus all_sites_seatbelt anchors

        self.anchors = generate_anchors(self.grid_size, self.input_size, self.anchors_seed)
        self.anchors_per_grid = self.anchors_seed.shape[0]
        self.num_anchors = self.anchors.shape[0]

        self.results_dir = os.path.join(cfg.save_dir, 'results')

    def get_sample_ids(self):
        #a dirty duct tape to load preprocessing of val phase but load dataset for train phase for trt calib
        sample_set_name = 'train.txt' if self.phase == 'train' \
            else 'train.txt' if self.phase == 'val' \
            else 'trainval.txt' if self.phase == 'trainval' \
            else None

        sample_ids_path = os.path.join(self.data_dir, 'image_sets', sample_set_name)
        with open(sample_ids_path, 'r') as fp:
            sample_ids = fp.readlines()
        sample_ids = tuple(x.strip() for x in sample_ids)

        return sample_ids, sample_ids_path

    def load_image(self, index):
        image_id = self.sample_ids[index]
        image_path = os.path.join(self.data_dir, 'training/image_2', image_id + '.jpg')
        image = default_loader(image_path)
        if image.mode == 'L':
            image = image.convert('RGB')
        image = np.array(image).astype(np.float32)
        # image = skimage.io.imread(image_path).astype(np.float32)
        return image, image_id

    def load_annotations(self, index):
        ann_id = self.sample_ids[index]
        ann_path = os.path.join(self.data_dir, 'training/label_2', ann_id + '.txt')
        with open(ann_path, 'r') as fp:
            annotations = fp.readlines()

        annotations = [ann.strip().split(' ') for ann in annotations]
        class_ids, boxes = [], []
        for ann in annotations:
            if ann[0] not in self.class_names:
                continue
            class_ids.append(self.class_ids_dict[ann[0]])
            box = [float(x) for x in ann[4:8]]
            # if box[2] <= 0:
            #     box[2] = 0.00001
            # if box[3] <= 0:
            #     box[3] = 0.00001
            boxes.append(box)

        class_ids = np.array(class_ids, dtype=np.int16)
        boxes = np.array(boxes, dtype=np.float32)
        if len(boxes):
            return class_ids, boxes
        boxes = None
        return class_ids, boxes

    # ========================================
    #                evaluation
    # ========================================

    def save_results(self, results):
        txt_dir = os.path.join(self.results_dir, 'data')
        os.makedirs(txt_dir, exist_ok=True)

        for res in results:
            txt_path = os.path.join(txt_dir, res['image_meta']['image_id'] + '.txt')
            if 'class_ids' not in res:
                with open(txt_path, 'w') as fp:
                    fp.write('')
                continue

            num_boxes = len(res['class_ids'])
            with open(txt_path, 'w') as fp:
                for i in range(num_boxes):
                    class_name = self.class_names[res['class_ids'][i]].lower()
                    score = res['scores'][i]
                    bbox = res['boxes'][i, :]
                    line = '{} -1 -1 0 {:.2f} {:.2f} {:.2f} {:.2f} 0 0 0 0 0 0 0 {:.3f}\n'.format(
                            class_name, *bbox, score)
                    fp.write(line)

    def evaluate(self):
        kitti_eval_tool_path = os.path.join(self.cfg.root_dir, 'src/utils/kitti-eval/cpp/evaluate_object')
        cmd = '{} {} {} {} {}'.format(kitti_eval_tool_path,
                                      os.path.join(self.data_dir, 'training'),
                                      self.sample_set_path,
                                      self.results_dir,
                                      len(self.sample_ids))

        status = subprocess.call(cmd, shell=True)

        aps = {}
        for class_name in self.class_names:
            map_path = os.path.join(self.results_dir, 'stats_{}_ap.txt'.format(class_name.lower()))
            if os.path.exists(map_path):
                with open(map_path, 'r') as f:
                    lines = f.readlines()
                _aps = [float(line.split('=')[1].strip()) for line in lines]
            else:
                _aps = [0., 0., 0.]

            aps[class_name + '_easy'] = _aps[0]
            aps[class_name + '_moderate'] = _aps[1]
            aps[class_name + '_hard'] = _aps[2]

        aps['mAP'] = sum(aps.values()) / len(aps)

        return aps



In [23]:
veh_det_calib_dataset = YOLO_CalibTRT('val', cfg)

In [24]:
calib_data_loader = torch.utils.data.DataLoader(veh_det_calib_dataset,
                                                  batch_size=cfg.batch_size,
                                                  num_workers=cfg.num_workers,
                                                  pin_memory=False)

In [25]:
# print (len(calib_data_loader))
# inp  = next(iter(calib_data_loader))
# print(inp['image'].shape)

# for i, inp in enumerate(calib_data_loader, 0):
#         print(inp['image'].shape)
#         if i >= 10:
#             break

In [ ]:
num_batches = int(len(veh_det_calib_dataset)/cfg.batch_size)
print (num_batches)
detector.model.eval()
detector.model.cuda()
with torch.no_grad():
    collect_stats(detector.model, calib_data_loader, num_batches=1000)
    # compute_amax(detector.model, method="percentile", percentile=99.99)

In [ ]:
print (detector.model)

In [16]:
from glob import glob
from os import *
from utils import image
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.transforms.functional import InterpolationMode
import cv2

images_dir = "/home/hazen/data/SqueezeDet-PyTorch/data/test_trt_2"
convdet_dir = "/home/hazen/data/SqueezeDet-PyTorch/data/wrong_convdet/"
orignal_dir = "/home/hazen/data/SqueezeDet-PyTorch/data/wrong_orignal/"
model = model.cuda()
image_paths = glob(path.join(images_dir,"*"))
image_paths.sort()
for i, path in enumerate(image_paths):
        # print(path)
        img = default_loader(path)
        #print(img)
        # input_image = np.array(img, dtype=np.float32, order='C')
        # if img.mode == 'L':
        #         img = img.convert('RGB')
        # print(np.array(img).shape)
        # gray_input = np.load(orignal_dir+path.split("/")[-1].split(".")[0]+".npy")
        # trt_xl = trt_xl/255
        # trt_xl = rgb2gray(trt_xl)
        # print(trt_xl.shape)
        # img = trt_xl.transpose(2, 0, 1)

        # trt_xl2 = np.load(orignal_dir+path.split("/")[-1].split(".")[0]+".npy")

        # print(np.max(img-trt_xl2))


        # print(trt_xl.shape)
        img = transforms.ToTensor()(img)
        
        img = transforms.Resize((256,448),interpolation=InterpolationMode.NEAREST) (img)
        
        # img = transforms.Grayscale(num_output_channels=3) (img)

        

        mean = np.array([0.41139729, 0.41338573, 0.41059459], dtype=np.float16).reshape(3, 1, 1)
        std = np.array([0.19880643, 0.19452718, 0.19670135], dtype=np.float16).reshape(3, 1, 1)

        img = (img-mean)/std

        convdet = np.load(convdet_dir+path.split("/")[-1].split(".")[0]+".npy")
        orig = np.load(orignal_dir+path.split("/")[-1].split(".")[0]+".npy")
        # print(orig.shape)
        print(torch.max(img-orig))

        # print(orig.shape)
        # x = img.numpy()-trt_xl
        # print(np.max(x))
        # # img = img.permute(2, 0, 1)
        # print(gray_input.shape)
        img = orig.reshape(1,3,256,448)
        # trt_xl = trt_xl.reshape(1,3,256,256)
        # x = img.numpy()-trt_xl
        # # print("2",np.max(x))
        img = torch.tensor(img,dtype=torch.float32)
        # img = torch.reshape(img, (1,3,256,256))
        # img = torch.rand(1,3,256,448)

        img = img.cuda()
        
        x,xl = model.base(img)
        # xl = xl.reshape(36,16,16)
        xl = xl.to('cpu').detach().numpy()
        # print(xl)
        # print("TRT: ",trt_xl)
        # print("PyTorch: ", xl)
        print(np.max(xl-convdet))

ValueError: operands could not be broadcast together with shapes (3,256,448) (1,1,3) 

In [36]:
model = torch.load("/home/hazen/data/test_weights/alpr_det.pth")
print(model['state_dict'].keys()) 

odict_keys(['base.features.0.weight', 'base.features.0.bias', 'base.features.3.squeeze.weight', 'base.features.3.squeeze.bias', 'base.features.3.expand1x1.weight', 'base.features.3.expand1x1.bias', 'base.features.3.expand3x3.weight', 'base.features.3.expand3x3.bias', 'base.features.4.squeeze.weight', 'base.features.4.squeeze.bias', 'base.features.4.expand1x1.weight', 'base.features.4.expand1x1.bias', 'base.features.4.expand3x3.weight', 'base.features.4.expand3x3.bias', 'base.features.6.squeeze.weight', 'base.features.6.squeeze.bias', 'base.features.6.expand1x1.weight', 'base.features.6.expand1x1.bias', 'base.features.6.expand3x3.weight', 'base.features.6.expand3x3.bias', 'base.features.7.squeeze.weight', 'base.features.7.squeeze.bias', 'base.features.7.expand1x1.weight', 'base.features.7.expand1x1.bias', 'base.features.7.expand3x3.weight', 'base.features.7.expand3x3.bias', 'base.features.9.squeeze.weight', 'base.features.9.squeeze.bias', 'base.features.9.expand1x1.weight', 'base.featur

### Try Different Scehmes and Eval Model

In [33]:
from datasets.yolo import YOLO

eval_dataset = YOLO('val', cfg)
eval_dataset.phase

'val'

In [34]:
with torch.no_grad():
    # for method in ["percentile", "mse", "entropy"]:
    for method in ["mse"]:
        percentile = 99.99
        print(F" ############  {method} calibration ######################")
        if(method == 'percentile'):
            compute_amax(detector.model, method=method, percentile = percentile)
        else:
            compute_amax(detector.model, method=method)
        update_exp_dir(cfg, f"trt_calib_veh_det_{method}")
        results = detector.detect_dataset(eval_dataset, cfg)
        eval_dataset.save_results(results)
        aps = eval_dataset.evaluate()
        for k, v in aps.items():
                print('{:<20} {:.3f}'.format(k, v))
        torch.save(detector.model.state_dict(), os.path.join(cfg.save_dir, f"quant_veh_det_squeezedet_{method}.pth"))

 ############  mse calibration ######################


W0818 16:11:01.860282 139799529214144 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0818 16:11:01.860819 139799529214144 tensor_quantizer.py:238] Call .cuda() if running on GPU after loading calibrated amax.
W0818 16:11:01.861232 139799529214144 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([64, 1, 1, 1]).
W0818 16:11:02.864593 139799529214144 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0818 16:11:02.865229 139799529214144 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([16, 1, 1, 1]).
W0818 16:11:05.107809 139799529214144 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0818 16:11:05.108374 139799529214144 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([64, 1, 1, 1]).
W0818 16:11:06.245361 139799529214144 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0818 16:11:06.246005 139799529214144 tensor_quantizer.py:237] Load calibrated amax, shape=to

eval: [0/7902] | data 0.051s | net 0.037s
eval: [20/7902] | data 0.089s | net 0.021s
eval: [40/7902] | data 0.046s | net 0.021s
eval: [60/7902] | data 0.059s | net 0.022s
eval: [80/7902] | data 0.074s | net 0.021s
eval: [100/7902] | data 0.093s | net 0.021s
eval: [120/7902] | data 0.075s | net 0.022s
eval: [140/7902] | data 0.094s | net 0.021s
eval: [160/7902] | data 0.104s | net 0.021s
eval: [180/7902] | data 0.106s | net 0.022s
eval: [200/7902] | data 0.086s | net 0.020s
eval: [220/7902] | data 0.105s | net 0.021s
eval: [240/7902] | data 0.090s | net 0.020s
eval: [260/7902] | data 0.084s | net 0.021s
eval: [280/7902] | data 0.087s | net 0.020s
eval: [300/7902] | data 0.074s | net 0.021s
eval: [320/7902] | data 0.104s | net 0.020s
eval: [340/7902] | data 0.083s | net 0.021s
eval: [360/7902] | data 0.106s | net 0.022s
eval: [380/7902] | data 0.073s | net 0.020s
eval: [400/7902] | data 0.070s | net 0.020s
eval: [420/7902] | data 0.071s | net 0.020s
eval: [440/7902] | data 0.090s | net 0

sh: 1: gnuplot: not found
sh: 1: gnuplot: not found
Error: /undefinedfilename in (bike_detection.eps)
Operand stack:

Execution stack:
   %interp_exit   .runexec2   --nostringval--   --nostringval--   --nostringval--   2   %stopped_push   --nostringval--   --nostringval--   --nostringval--   false   1   %stopped_push
Dictionary stack:
   --dict:956/1684(ro)(G)--   --dict:0/20(G)--   --dict:77/200(L)--
Current allocation mode is local
Last OS error: No such file or directory
GPL Ghostscript 9.26: Unrecoverable error, exit code 1
sh: 1: pdfcrop: not found
sh: 1: gnuplot: not found
sh: 1: gnuplot: not found
Error: /undefinedfilename in (bike_orientation.eps)
Operand stack:

Execution stack:
   %interp_exit   .runexec2   --nostringval--   --nostringval--   --nostringval--   2   %stopped_push   --nostringval--   --nostringval--   --nostringval--   false   1   %stopped_push
Dictionary stack:
   --dict:956/1684(ro)(G)--   --dict:0/20(G)--   --dict:77/200(L)--
Current allocation mode is local


Thank you for participating in our evaluation!
Loading detections...
  done.
Your evaluation results are available in /home/gj/hazen/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_veh_det_trt_calib_sqdet/results
bike_easy            0.525
bike_moderate        0.525
bike_hard            0.525
car_easy             0.765
car_moderate         0.762
car_hard             0.762
bus_easy             0.489
bus_moderate         0.488
bus_hard             0.488
mAP                  0.592


sh: 1: gnuplot: not found
sh: 1: gnuplot: not found
Error: /undefinedfilename in (bus_detection.eps)
Operand stack:

Execution stack:
   %interp_exit   .runexec2   --nostringval--   --nostringval--   --nostringval--   2   %stopped_push   --nostringval--   --nostringval--   --nostringval--   false   1   %stopped_push
Dictionary stack:
   --dict:956/1684(ro)(G)--   --dict:0/20(G)--   --dict:77/200(L)--
Current allocation mode is local
Last OS error: No such file or directory
GPL Ghostscript 9.26: Unrecoverable error, exit code 1
sh: 1: pdfcrop: not found
sh: 1: gnuplot: not found
sh: 1: gnuplot: not found
Error: /undefinedfilename in (bus_orientation.eps)
Operand stack:

Execution stack:
   %interp_exit   .runexec2   --nostringval--   --nostringval--   --nostringval--   2   %stopped_push   --nostringval--   --nostringval--   --nostringval--   false   1   %stopped_push
Dictionary stack:
   --dict:956/1684(ro)(G)--   --dict:0/20(G)--   --dict:77/200(L)--
Current allocation mode is local
La

'val'

In [ ]:
# # print (detector.model)
# print (detector.model.base.features[0][0].input_quantizer.amax)
# print (detector.model.base.features[0][0].weight_quantizer.amax)

In [ ]:
# eval_dataset.save_results(results)
# aps = eval_dataset.evaluate()
# for k, v in aps.items():
#         print('{:<20} {:.3f}'.format(k, v))

In [35]:
torch.save(detector.model.state_dict(), "../exp/quant_veh_det_squeezedet_model_41.pth")

In [48]:
# print (dir(detector.model.base.conv1.weight_quantizer))
# print (detector.model.base.conv1.weight_quantizer.scale)

E0818 16:44:35.580255 139799529214144 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!


1.0


In [ ]:
# test = detector.model.base
# test.cpu()
# quant_nn.TensorQuantizer.use_fb_fake_quant = True
# dummy_input = torch.randn(1, 3, 128, 128)

# # input_names = [ "network.0" ]
# # output_names = [ "network.23" ]

# # enable_onnx_checker needs to be disabled. See notes below.
# torch.onnx.export(
#     test, dummy_input, "../exp/test.onnx", verbose=True, opset_version=13, enable_onnx_checker=False)

In [2]:
dict = torch.load("../exp/quant_veh_det_squeezedet_model_41.pth")
print (dict.keys())

odict_keys(['base.conv1.weight', 'base.conv1.bias', 'base.conv1._input_quantizer._amax', 'base.conv1._weight_quantizer._amax', 'base.features.1.squeeze.weight', 'base.features.1.squeeze.bias', 'base.features.1.squeeze._input_quantizer._amax', 'base.features.1.squeeze._weight_quantizer._amax', 'base.features.1.expand1x1.weight', 'base.features.1.expand1x1.bias', 'base.features.1.expand1x1._input_quantizer._amax', 'base.features.1.expand1x1._weight_quantizer._amax', 'base.features.1.expand3x3.weight', 'base.features.1.expand3x3.bias', 'base.features.1.expand3x3._input_quantizer._amax', 'base.features.1.expand3x3._weight_quantizer._amax', 'base.features.2.squeeze.weight', 'base.features.2.squeeze.bias', 'base.features.2.squeeze._input_quantizer._amax', 'base.features.2.squeeze._weight_quantizer._amax', 'base.features.2.expand1x1.weight', 'base.features.2.expand1x1.bias', 'base.features.2.expand1x1._input_quantizer._amax', 'base.features.2.expand1x1._weight_quantizer._amax', 'base.features

In [3]:
# print (alpr)
def get_scales_input_per_tensor(amax, _num_bits = 8 , _unsigned = False):
    bound = (1 << (_num_bits - 1 + int(_unsigned))) - 1    
    zero_point_per_tensor = 0 #since symmertric quant
    scale_per_tensor = amax / bound
    return scale_per_tensor, zero_point_per_tensor

def get_scales_weight_per_channel(amax, _num_bits = 8, _unsigned = False):
    bound = (1 << (_num_bits - 1 + int(_unsigned))) - 1
    amax_sequeeze = amax.squeeze().detach()
    scale_per_channel =  amax_sequeeze/bound
    zero_point_per_channel = torch.zeros_like(scale_per_channel, dtype=torch.int32).data
    quant_axis = list(amax.shape).index(list(amax_sequeeze.shape)[0])
    return scale_per_channel, zero_point_per_channel


In [4]:
veh_det_epoch41_dict_with_scales = {}

for key in dict:
    if key.find('_input_quantizer._amax') != -1:
        amax = dict[key]
        scales, zero_point = get_scales_input_per_tensor(amax)
        new_key = key + '.scales'
        veh_det_epoch41_dict_with_scales[new_key] = scales
    elif key.find('_weight_quantizer._amax') != -1:
        amax = dict[key]
        scales, zero_point = get_scales_weight_per_channel(amax)
        new_key = key + '.scales'
        veh_det_epoch41_dict_with_scales[new_key] = scales
    
    veh_det_epoch41_dict_with_scales[key] = dict[key]

print (veh_det_epoch41_dict_with_scales.keys())
# print (veh_det_epoch41_dict_with_scales['conv1._input_quantizer._amax.scales'])

dict_keys(['base.conv1.weight', 'base.conv1.bias', 'base.conv1._input_quantizer._amax.scales', 'base.conv1._input_quantizer._amax', 'base.conv1._weight_quantizer._amax.scales', 'base.conv1._weight_quantizer._amax', 'base.features.1.squeeze.weight', 'base.features.1.squeeze.bias', 'base.features.1.squeeze._input_quantizer._amax.scales', 'base.features.1.squeeze._input_quantizer._amax', 'base.features.1.squeeze._weight_quantizer._amax.scales', 'base.features.1.squeeze._weight_quantizer._amax', 'base.features.1.expand1x1.weight', 'base.features.1.expand1x1.bias', 'base.features.1.expand1x1._input_quantizer._amax.scales', 'base.features.1.expand1x1._input_quantizer._amax', 'base.features.1.expand1x1._weight_quantizer._amax.scales', 'base.features.1.expand1x1._weight_quantizer._amax', 'base.features.1.expand3x3.weight', 'base.features.1.expand3x3.bias', 'base.features.1.expand3x3._input_quantizer._amax.scales', 'base.features.1.expand3x3._input_quantizer._amax', 'base.features.1.expand3x3._

In [5]:
torch.save(veh_det_epoch41_dict_with_scales, "../exp/quant_veh_det_squeezedet_model_41_w_scales.pth")

In [6]:
# a = torch.load("../exp/quant_veh_det_squeezedet_model_41_w_scales.pth")
# a.keys()

dict_keys(['base.conv1.weight', 'base.conv1.bias', 'base.conv1._input_quantizer._amax.scales', 'base.conv1._input_quantizer._amax', 'base.conv1._weight_quantizer._amax.scales', 'base.conv1._weight_quantizer._amax', 'base.features.1.squeeze.weight', 'base.features.1.squeeze.bias', 'base.features.1.squeeze._input_quantizer._amax.scales', 'base.features.1.squeeze._input_quantizer._amax', 'base.features.1.squeeze._weight_quantizer._amax.scales', 'base.features.1.squeeze._weight_quantizer._amax', 'base.features.1.expand1x1.weight', 'base.features.1.expand1x1.bias', 'base.features.1.expand1x1._input_quantizer._amax.scales', 'base.features.1.expand1x1._input_quantizer._amax', 'base.features.1.expand1x1._weight_quantizer._amax.scales', 'base.features.1.expand1x1._weight_quantizer._amax', 'base.features.1.expand3x3.weight', 'base.features.1.expand3x3.bias', 'base.features.1.expand3x3._input_quantizer._amax.scales', 'base.features.1.expand3x3._input_quantizer._amax', 'base.features.1.expand3x3._

In [8]:
c = torch.load('../exp/model_41.pth')
print (c['state_dict'].keys())

odict_keys(['base.features.0.weight', 'base.features.0.bias', 'base.features.3.squeeze.weight', 'base.features.3.squeeze.bias', 'base.features.3.expand1x1.weight', 'base.features.3.expand1x1.bias', 'base.features.3.expand3x3.weight', 'base.features.3.expand3x3.bias', 'base.features.4.squeeze.weight', 'base.features.4.squeeze.bias', 'base.features.4.expand1x1.weight', 'base.features.4.expand1x1.bias', 'base.features.4.expand3x3.weight', 'base.features.4.expand3x3.bias', 'base.features.6.squeeze.weight', 'base.features.6.squeeze.bias', 'base.features.6.expand1x1.weight', 'base.features.6.expand1x1.bias', 'base.features.6.expand3x3.weight', 'base.features.6.expand3x3.bias', 'base.features.7.squeeze.weight', 'base.features.7.squeeze.bias', 'base.features.7.expand1x1.weight', 'base.features.7.expand1x1.bias', 'base.features.7.expand3x3.weight', 'base.features.7.expand3x3.bias', 'base.features.9.squeeze.weight', 'base.features.9.squeeze.bias', 'base.features.9.expand1x1.weight', 'base.featur